# Práctica 2

## Minería de Datos 2016/2017 - Rubén Donate Serrano

In [1]:
# Antes de empezar es fundamental cargar las librerias de python!
import numpy as np
import pandas as pd
from ggplot import *
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)
# Configuracion para seaborn
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

from sklearn import neighbors
from sklearn import tree
from sklearn.cross_validation import train_test_split
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA 

# Imprimir un arbol

import pydotplus 
from IPython.display import Image  

def imprimeArbol(model, data, width):
    fnames = data.columns.values[:-1]
    lnames = data['label'].unique()
    dot_data = tree.export_graphviz(model, out_file=None, 
                             feature_names=fnames,  
                             class_names=lnames,  
                             filled=True, rounded=True,  
                             special_characters=True)  
    graph = pydotplus.graph_from_dot_data(dot_data)  
    return Image(graph.create_png(), width=width)

/usr/local/lib/python2.7/dist-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/usr/local/lib/python2.7/dist-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interf

Realizamos la importacion de todos los paquetes que van a ser necesarios para la resolucion de la practica y las funciones que se son han facilitado.

### Preprocesador de los datos

In [2]:
def limpiaDataSet (df):
    df_limpio = df.dropna(axis=0)
    return df_limpio
def separarAtributosClase(df,etiqueta):
    df_limpio = limpiaDataSet(df)
    atributos = df_limpio.drop(etiqueta, 1)
    clase = df_limpio[etiqueta]
    return atributos, clase

En este punto se procede a realizar un preprocesado de un DataSet generico, el cual se pasara mediante parametros a la funcion junto con el valor de la etiqueta que identifica a los valores de la clase, consistente en separar los atributos de los valores de la clase. Antes de esto, se realiza un limpieza generica, para lo cual se le procedera a pasarle como parametro del DataSet, del mismo eliminando los registros que poseen valores perdidos.

### Envio informacion al servidor.

In [3]:
import pycurl, io, json

semilla = 3955
df_val = pd.read_csv("./data/titanic_validacion.csv")
df_val = df_val.drop("Name", 1)
df_val = df_val.drop("PassengerId",1)
df_train = pd.read_csv("./data/titanic_train.csv")
df_train = df_train.drop("Name", 1)
df_train = df_train.drop("PassengerId",1)
atrib_train , clase_train = separarAtributosClase(df_train,"label")

def envioPrediccion(prediccion):
    user       = 'Ruben.Donate@alu.uclm.es'
    passwd     = 'TNRQB7ZO'
    # Este código recibe directamente el vector de predicciones y lo convierte a una
    # String separando los valores por comas
    pred_str   = ','.join(prediccion.tolist())
 
    url  = "http://212.47.226.96/predict"
    data = json.dumps({'user':user, 'passwd':passwd, 'prediction':pred_str})
    
    out = io.BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(pycurl.HTTPHEADER, ['Content-Type: application/json'])
    c.setopt(pycurl.POST, 1)
    c.setopt(c.POSTFIELDS, data)
    c.setopt(c.WRITEFUNCTION, out.write)
    
    c.perform()
    
    response = json.loads(out.getvalue().decode())
    print("Response: ",response)
    out.close()

En este punto, se realiza las importaciones y la funcion personalizada en mi caso para el envio de la predicion al servidor para su validacion. En esta personalizacion realizado una limpieza de los valores perdidos de los valores necesarios para realizar el entrenamiento. A continuación, se separan los atributos predictores de los valores de la clase para proceder a entrenar el modelo y a partir de este punto realizar la prediccion. Esta prediccion se procede a su envio con el usuario y la contraseña que se me ha facilitado.

### Variables

In [4]:
vecinos={}
arboles={}

modeloMejorVecino=()

modeloMejorArbol=()

Tener almacenada toda la información que se ha generado he decidido utilizar un diccionario y una variable con el mejor valor para cada uno de los modelos que se ha tenido que implementar en el algoritmo. Esto lo he realiza de este modo para que sea posible mostar los resultados obtenidos en cada uno de los modelos generados. Por ese motivo en cualquiera de los casos un elemento sería una tupla con una identificación del modelo, en el knn sería el valor de k y para los arboles una tupla con los parametros que se ha utilizado para su configuracion, y otro elemento que es otraa tupla con el modelo generado, los valores obtenidos a traves del cross validation y la media del mismo.

### Comparacion

In [5]:
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1]) for x in list])
    display(HTML(report))

def printTable2(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1] + cell % x[2] + cell % x[3] + cell % x[4]) for x in list])
    display(HTML(report))

### Carga de los Dataset de Training

En este punto, se genera una lista de DataSet a los cuales se le realizara una eliminacion de los valores que no tiene significado para la clasificacion que se pueden observar a simple vista, por ejemplo los identificadores o los nombre que identifican a un elemento.

#### Base de Datos Iris

In [6]:
irisdf = pd.read_csv("./data/iris.csv")

#### Base de Datos Pima

In [7]:
pimadf = pd.read_csv("./data/pima.csv")

#### Base de Datos Wisconsin

In [8]:
wisconsindf = pd.read_csv("./data/wisconsin.csv")
wisconsindf = wisconsindf.drop("patientId", 1)

#### Base de Datos de Titanic

In [9]:
titanicdf = df_train

En un principio, tenia pensado realizar las pruebas con cada uno de los DataSet. Pero al final, solo trabaje con este DataSet, el cual se recupera del proceso de envio una vez limpiado.

### Cross Validation

In [10]:
def crossValidation(modelo,df,etiqueta,partes,medida,semilla):
    np.random.seed(seed=semilla)
    atributos, clase = separarAtributosClase(df,etiqueta)
    scores = cross_val_score(modelo, atributos, clase, cv=partes, scoring=medida)
    return scores

Ahora lo que realizamos es declarar una funcion al cual se encargara de realizar la validacion cruzada y devolvera los valores que se han obtenido para ese caso.

### Modelos por defecto

A continuacion, procedere a crear las funciones para generar los modelos con los parametros de manera generica, las cuales devolverán la tupla con el modelo, los valores obtenidos en la validacion cruzada y la media de estos valores.

#### KNN

In [11]:
def modelokNN (k,df,etiqueta,partes,medida,seed):
    modelo = neighbors.KNeighborsClassifier(k)
    metrica = crossValidation(modelo,df,etiqueta,partes,medida,seed)
    return (modelo,metrica,metrica.mean())

Para el knn lo que se ha realizado es paraserle a la funcion todos los valores necesarios para generar el modelo, en este caso el valor de la k, y para realizar la validacion cruzada.

#### Arbol de Decision

In [12]:
def modeloArbol (df,etiqueta,criterio,profundidad,minEjemplos,poda,partes,medida,seed):
    modelo= tree.DecisionTreeClassifier(criterion = criterio, max_depth = profundidad, min_samples_split = minEjemplos, min_impurity_split = poda, random_state = seed)
    metrica = crossValidation(modelo,df,etiqueta,partes,medida,seed)
    return (modelo,metrica,metrica.mean())

Del mismo modo que en el caso del knn, lo que se ha realizado es pasar a la funcion arbol todos los parametros necesarios para configurar el arbol, los cuales son criterio que establece la medida a partir de la cual se generara el arbol, profundidad que podra alcanzar el arbol, numero minimo de ejemplos que minimo de instancias a partir del cual no ramifica, poda que es el valor a partir de la ganancia de informacion del nodo, el numero aleatorio.

### Selector de modelo

In [13]:
seed = 3568
k = 2
etiqueta = "label"
medida = "accuracy"
partes = 10
df = titanicdf
knnPrevio = modelokNN ((k-1),df,etiqueta,partes,medida,seed)
vecinos[(k-1)]=knnPrevio
modeloMejorVecino = knnPrevio
knn = modelokNN (k,df,etiqueta,partes,medida,seed)
vecinos[k]=knn
while (abs(knnPrevio[2]-knn[2]))>0.0001:
    if (knn[2]>modeloMejorVecino[2]):
        modeloMejorVecino = knn
    knnPrevio = knn
    k+=1
    knn = modelokNN (k,df,etiqueta,partes,medida,seed)
    vecinos[k]=knn
if (knn[2]>modeloMejorVecino[2]):
    modeloMejorVecino = knn

El algoritmo implementado consiste realizar un bucle si el valor absoluto de media del modelo para k-1 menos el de k es mayor de 0.0001 eso quiere decir que el modelo de k respecto del anterior empeora o mejora un poco por lo cual se de para y se selecciona el valor mejor que se ha obtenido hasta el momento. El valor de k en cada iteraccion del bucle se incrementa en uno.

In [14]:
seed = 3568
etiqueta = "label"
medida = "accuracy"
partes = 10
df = titanicdf
criterio = ("entropy","gini")
profundidad = None
minEjemplos = 2
poda = 1e-7
modeloMejorArbol = modeloArbol (df,etiqueta,criterio[0],profundidad,minEjemplos,poda,partes,medida,seed)
clasificador = modeloMejorArbol[0].fit(atrib_train , clase_train)
min_profundidad = 1
max_profundidad = clasificador.tree_.max_depth
arboles[(criterio[0],max_profundidad,minEjemplos,poda)]=modeloMejorArbol
profundidad =int((max_profundidad-min_profundidad)/2)
arbol=modeloArbol(df,etiqueta,criterio[0],profundidad,minEjemplos,poda,partes,medida,seed)
arboles[(criterio[0],profundidad,minEjemplos,poda)]=arbol
while ((modeloMejorArbol[2]<arbol[2])and(max_profundidad-min_profundidad>=1)):
    modeloMejorArbol=arbol
    max_profundidad=profundidad
    profundidad =int((max_profundidad-min_profundidad)/2)
    for c in criterio:
        aux_arbol1=modeloArbol(df,etiqueta,c,profundidad,minEjemplos,poda,partes,medida,seed)
        arboles[(c,profundidad,minEjemplos,poda)]=aux_arbol1
        if aux_arbol1[2]>=arbol[2]:
            arbol=aux_arbol1
    if ((modeloMejorArbol[2]>=arbol[2])and(max_profundidad-profundidad>=0)):
        min_profundidad = profundidad
        profundidad =int((max_profundidad-min_profundidad)/2)
        for c in criterio:
            aux_arbol2=modeloArbol(df,etiqueta,c,profundidad,minEjemplos,poda,partes,medida,seed)
            arboles[(c,profundidad,minEjemplos,poda)]=aux_arbol2
            if aux_arbol2[2]>=arbol[2]:
                arbol=aux_arbol2

El algoritmo de selección de atributos lo he centrado en obtener la profundidad optima y para los dos criterios de evaluación. Para realizarlo se ha entrenado un arbol con los valores por defecto para obtener una profundidad de partida que se considerara como maxima. Con este valor y un minimo de uno se ha implementado un algoritmo de bisiceccion para obtener el valor de la manera mas eficaz.

In [15]:
if (modeloMejorVecino[2]>modeloMejorArbol[2]):
    clasificador = modeloMejorVecino[0].fit(atrib_train , clase_train)
else:
    clasificador = modeloMejorArbol[0].fit(atrib_train , clase_train)

In [16]:
prediccion = clasificador.predict(df_val)

In [17]:
listaAccuracyVecinos=[("Número Vecinos", "Accuracy")]
for vecino in vecinos:
    listaAccuracyVecinos.append((vecino,vecinos[vecino][2]))
printTable(listaAccuracyVecinos)

Número Vecinos,Accuracy
1,0.647558790594
2,0.646539753639
3,0.663393057111
4,0.666550951848
5,0.655968645017
6,0.669832026876
7,0.659227323628
8,0.662463605823
9,0.66241881299


In [18]:
listaAccuracyArboles=[("Criterio","Profundidad","Minimo Ejemplos","Poda", "Accuracy"),]
for tree in arboles:
    listaAccuracyArboles.append((tree[0],tree[1],tree[2],tree[3],arboles[tree][2]))
printTable2(listaAccuracyArboles)

Criterio,Profundidad,Minimo Ejemplos,Poda,Accuracy
gini,5,2,1e-07,0.79477043673
entropy,23,2,1e-07,0.754423292273
entropy,2,2,1e-07,0.786203807391
entropy,11,2,1e-07,0.777737961926
entropy,1,2,1e-07,0.784132138858
entropy,5,2,1e-07,0.803146696529
gini,1,2,1e-07,0.784132138858
gini,2,2,1e-07,0.785139977604


In [19]:
#envioPrediccion(prediccion)